In [1]:
import pandas as pd
import geopandas as gpd
import pickle

import fiona

---
# Nuevo enfoque
Dado a que nuestro peculiar interes es concervar la intregridad de los datos originales, y poder acompletar el 100 porciento de los mismos con exito,

Considerando que los datos de entidades y CP que contiene el GPKG estan actulizados al 2020, y que posterior a este aun no se publican actulizaciones, esto nos motiva a proceder de la siguiente manra:

* Considerar que tan solo en el 2022 hubo alredodor de 837 cambios de CP a nivel nacional, nos ayudara a no confiarnos de los CP como puntos de comparación.
* Primero generar la comparación con los y match de los datos a partir de la similitud de sus colonias y entidades, asignar un CP a forma de etiqueta.
* Con CP a forma de etiqueta coparar con los CP de los datos originales, pero en caso de que sean diferentes no se descartan, pero no ayudara para mas adelate asignar un indice de marginalidad.
* Jerarquia con la que se buscara hacer match para llenar y corregir los datos: 
  * Con el data frame del indice de marginalidad.
  * Con el catalogo del 2025 
  * Comparar por punto por su api
   

In [2]:
DIR_GPKG = '../DataSets_Iniciales/colonias_indice_marginacion_cdmx/indice_marginacion_colonias_valle_mexico.gpkg'

In [3]:
name_layer = fiona.listlayers(DIR_GPKG)
gdf = gpd.read_file(DIR_GPKG, layer=name_layer[0])

if type(gdf)== gpd.GeoDataFrame:
    df_indice_marginacion = pd.DataFrame(gdf)

df_indice_marginacion

,GM_2020,COLONIA,CP,NOM_MUN,NOM_ENT,geometry
0,Bajo,aguilera,2900.0,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15443 19.47346, -99.15616 ..."
1,Bajo,aldana,2910.0,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1461 19.47323, -99.1473 19..."
2,Muy bajo,ampliacion del gas,2970.0,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15522 19.46844, -99.15634 ..."
3,Bajo,ampliacion petrolera,2470.0,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1965 19.4852, -99.19667 19..."
4,Medio,ampliacion san pedro xalpa,2719.0,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.21601 19.48404, -99.21667 ..."
...,...,...,...,...,...,...
5955,Bajo,jajalpa,54980.0,tultepec,edomex,"MULTIPOLYGON (((-99.1241 19.6583, -99.12502 19..."
5956,Medio,la providencia,54980.0,tultepec,edomex,"MULTIPOLYGON (((-99.13675 19.64888, -99.13642 ..."
5957,Bajo,santiago teyahualco,54980.0,tultepec,edomex,"MULTIPOLYGON (((-99.11802 19.66253, -99.11865 ..."
5958,Medio,ex hda la mariscala,54949.0,tultitlan,edomex,"MULTIPOLYGON (((-99.13577 19.62349, -99.13583 ..."


In [6]:
import numpy as np

# Get column names as a list
column_names = df_indice_marginacion.columns.tolist()
print(column_names)

# If you need it as a numpy array instead
column_names_array = np.array(df_indice_marginacion.columns)
print(column_names_array)

['GM_2020', 'COLONIA', 'CP', 'NOM_MUN', 'NOM_ENT', 'geometry']
['GM_2020' 'COLONIA' 'CP' 'NOM_MUN' 'NOM_ENT' 'geometry']


In [4]:
# Porcentaje con formato
(df_indice_marginacion['GM_2020'].value_counts(normalize=True) * 100).apply(lambda x: f"{x:.2f}%")

GM_2020
Bajo        38.96%
Medio       32.45%
Muy bajo    12.99%
Alto        12.60%
Muy alto     3.00%
Name: proportion, dtype: object

In [5]:
df_indice_marginacion['NOM_ENT'].value_counts()

NOM_ENT
edomex    3605
cdmx      2243
Name: count, dtype: int64

---
